# Keras / TensorFlow Playground

https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html

I used the code from the link above to get started.

In [1]:
import tensorflow as tf
sess = tf.Session() #creates the tensorflow 'instance'

from keras import backend as K
K.set_session(sess) #associates the tensorflow instance created above with Keras

Using TensorFlow backend.


### After creating the tensorflow instance and pointing Keras to it, we build the network. First, we'll create an input and label 'placeholder' so we can proceed with the network before loading the image data ( I assume ).

In [6]:
# this placeholder will contain our input digits, as flat vectors
img = tf.placeholder(tf.float32, shape=(None, 784))
# img is used in the first layer below, labels is used one cell further

# this placeholder is for the labels
labels = tf.placeholder(tf.float32, shape=(None, 10))

In [7]:
from keras.layers import Dense

# Keras layers can be called on TensorFlow tensors:
x = Dense(128, activation='relu')(img) 
# fully-connected layer with 128 units and ReLU activation
x = Dense(128, activation='relu')(x)
preds = Dense(10, activation='softmax')(x) 
# output layer with 10 units and a softmax activation

In [8]:
# Defining the loss function

from keras.objectives import categorical_crossentropy
loss = tf.reduce_mean(categorical_crossentropy(labels, preds))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


# Training the model with a TensorFlow optimizer:

This is where you would insert your own images to train a model for a different purpose,
ie object detection.

In [9]:
from tensorflow.examples.tutorials.mnist import input_data
mnist_data = input_data.read_data_sets('MNIST_data', one_hot=True)

train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

# Initialize all variables
init_op = tf.global_variables_initializer()
sess.run(init_op)

# Run training loop
with sess.as_default():
    for i in range(100):
        batch = mnist_data.train.next_batch(50)
        train_step.run(feed_dict={img: batch[0],
                                  labels: batch[1]})

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
